In [1]:
import pandas as pd

In [3]:
train = pd.read_csv("../data/liver-data-processed/train.csv")
dev = pd.read_csv("../data/liver-data-processed/devSet.csv")
test = pd.read_csv("../data/liver-data-processed/test.csv")

data = pd.concat([train, dev, test])

data.REGION = data.REGION.astype(int)
cron = data.groupby("REGION")["DAYSWAIT_CHRON"].mean().to_frame().reset_index()
cron.rename(columns = {"DAYSWAIT_CHRON": "WaitingTime"}, inplace = True)
print cron
data = data.loc[:, ("REGION", "GSTATUS", "GTIME")]

    REGION  WaitingTime
0        1   256.817578
1        2   208.259302
2        3   120.151769
3        4   247.150773
4        5   299.279828
5        6   214.844840
6        7   209.787641
7        8   219.258299
8        9   257.963572
9       10   159.466566
10      11   172.413258


In [5]:
three_months = data.loc[data["GTIME"] <= 90]
print three_months.shape
three_months = three_months.drop(columns = ["GTIME"])
three_months.rename(columns = {"GSTATUS": "status_3_m"}, inplace = True)
print three_months.head(10)

(10765, 3)
    REGION  status_3_m
21       3           1
25       3           1
34       3           1
44       7           1
47       5           1
51       2           0
53      11           1
55       9           1
58      10           1
67       3           1


In [7]:
six_months = data.loc[data["GTIME"] <= 180]
print six_months.shape
six_months = six_months.drop(columns = ["GTIME"])
six_months.rename(columns = {"GSTATUS": "status_6_m"}, inplace = True)
print six_months.head(10)

(14804, 3)
    REGION  status_6_m
21       3           1
25       3           1
34       3           1
40       8           0
44       7           1
47       5           1
51       2           0
53      11           1
55       9           1
58      10           1


In [8]:
one_year = data.loc[data["GTIME"] <= 365]
print one_year.shape
one_year = one_year.drop(columns = ["GTIME"])
one_year.rename(columns = {"GSTATUS": "status_1_y"}, inplace = True)
print one_year.head(5)

(23952, 3)
    REGION  status_1_y
2        6           0
5        2           1
6        9           0
9        4           0
21       3           1


In [9]:
three_years = data.loc[data["GTIME"] <= 3 * 365]
print three_years.shape
three_years = three_years.drop(columns = ["GTIME"])
three_years.rename(columns = {"GSTATUS": "status_3_y"}, inplace = True)
print three_years.head(5)

(44697, 3)
   REGION  status_3_y
2       6           0
3       2           1
5       2           1
6       9           0
7       3           1


In [10]:
three_months = three_months.groupby("REGION").mean().reset_index()
three_months["status_3_m"] = three_months["status_3_m"] * 100
print three_months

    REGION  status_3_m
0        1   71.246819
1        2   72.705882
2        3   59.396985
3        4   66.704675
4        5   66.258503
5        6   63.761468
6        7   74.568966
7        8   72.596154
8        9   80.303030
9       10   69.019608
10      11   67.715827


In [11]:
six_months = six_months.groupby("REGION").mean().reset_index()
six_months["status_6_m"] = six_months["status_6_m"] * 100
print six_months

    REGION  status_6_m
0        1   65.734266
1        2   71.500560
2        3   57.833583
3        4   61.217949
4        5   59.569490
5        6   58.196721
6        7   74.139311
7        8   68.505747
8        9   79.629630
9       10   66.447368
10      11   67.328767


In [12]:
one_year = one_year.groupby("REGION").mean().reset_index()
one_year["status_1_y"] = one_year["status_1_y"] * 100
print one_year

    REGION  status_1_y
0        1   52.947720
1        2   58.707301
2        3   49.461862
3        4   48.983543
4        5   48.828676
5        6   41.114458
6        7   62.449393
7        8   55.013928
8        9   70.106316
9       10   54.365437
10      11   54.394403


In [13]:
three_years = three_years.groupby("REGION").mean().reset_index()
three_years["status_3_y"] = three_years["status_3_y"] * 100
print three_years

    REGION  status_3_y
0        1   42.414665
1        2   47.970144
2        3   40.472323
3        4   39.916482
4        5   38.202078
5        6   35.294118
6        7   48.834717
7        8   41.923905
8        9   58.103326
9       10   44.545916
10      11   43.403094


In [15]:
regions = pd.read_csv("../data/wage_gap_all.csv")
regions = regions.loc[:, ["Region", "State"]]
regions.columns = ["REGION", "STATE"]
print regions.loc[:10, :]

    REGION STATE
0        1    RI
1        1    MA
2        1    CT
3        1    ME
4        1    NH
5        2    DE
6        2    DC
7        2    MD
8        2    NJ
9        2    PA
10       2    WV


In [19]:
merged_data = pd.merge(three_months, regions, on="REGION")
merged_data = pd.merge(merged_data, six_months, on="REGION")
merged_data = pd.merge(merged_data, one_year, on="REGION")
merged_data = pd.merge(merged_data, three_years, on="REGION")
merged_data = pd.merge(merged_data, cron, on="REGION")
print merged_data

    REGION  status_3_m STATE  status_6_m  status_1_y  status_3_y  WaitingTime
0        1   71.246819    RI   65.734266   52.947720   42.414665   256.817578
1        1   71.246819    MA   65.734266   52.947720   42.414665   256.817578
2        1   71.246819    CT   65.734266   52.947720   42.414665   256.817578
3        1   71.246819    ME   65.734266   52.947720   42.414665   256.817578
4        1   71.246819    NH   65.734266   52.947720   42.414665   256.817578
5        1   71.246819    VT   65.734266   52.947720   42.414665   256.817578
6        2   72.705882    DE   71.500560   58.707301   47.970144   208.259302
7        2   72.705882    DC   71.500560   58.707301   47.970144   208.259302
8        2   72.705882    MD   71.500560   58.707301   47.970144   208.259302
9        2   72.705882    NJ   71.500560   58.707301   47.970144   208.259302
10       2   72.705882    PA   71.500560   58.707301   47.970144   208.259302
11       2   72.705882    WV   71.500560   58.707301   47.970144

In [20]:
merged_data = merged_data[["REGION", "STATE", "status_3_m", "status_6_m", "status_1_y", "status_3_y", "WaitingTime"]]
print merged_data

    REGION STATE  status_3_m  status_6_m  status_1_y  status_3_y  WaitingTime
0        1    RI   71.246819   65.734266   52.947720   42.414665   256.817578
1        1    MA   71.246819   65.734266   52.947720   42.414665   256.817578
2        1    CT   71.246819   65.734266   52.947720   42.414665   256.817578
3        1    ME   71.246819   65.734266   52.947720   42.414665   256.817578
4        1    NH   71.246819   65.734266   52.947720   42.414665   256.817578
5        1    VT   71.246819   65.734266   52.947720   42.414665   256.817578
6        2    DE   72.705882   71.500560   58.707301   47.970144   208.259302
7        2    DC   72.705882   71.500560   58.707301   47.970144   208.259302
8        2    MD   72.705882   71.500560   58.707301   47.970144   208.259302
9        2    NJ   72.705882   71.500560   58.707301   47.970144   208.259302
10       2    PA   72.705882   71.500560   58.707301   47.970144   208.259302
11       2    WV   72.705882   71.500560   58.707301   47.970144

In [22]:
merged_data["status_3_m"] = 100 - merged_data["status_3_m"]
merged_data["status_6_m"] = 100 - merged_data["status_6_m"]
merged_data["status_1_y"] = 100 - merged_data["status_1_y"]
merged_data["status_3_y"] = 100 - merged_data["status_3_y"]
print merged_data

    REGION STATE  status_3_m  status_6_m  status_1_y  status_3_y  WaitingTime
0        1    RI   28.753181   34.265734   47.052280   57.585335   256.817578
1        1    MA   28.753181   34.265734   47.052280   57.585335   256.817578
2        1    CT   28.753181   34.265734   47.052280   57.585335   256.817578
3        1    ME   28.753181   34.265734   47.052280   57.585335   256.817578
4        1    NH   28.753181   34.265734   47.052280   57.585335   256.817578
5        1    VT   28.753181   34.265734   47.052280   57.585335   256.817578
6        2    DE   27.294118   28.499440   41.292699   52.029856   208.259302
7        2    DC   27.294118   28.499440   41.292699   52.029856   208.259302
8        2    MD   27.294118   28.499440   41.292699   52.029856   208.259302
9        2    NJ   27.294118   28.499440   41.292699   52.029856   208.259302
10       2    PA   27.294118   28.499440   41.292699   52.029856   208.259302
11       2    WV   27.294118   28.499440   41.292699   52.029856

In [23]:
merged_data.to_csv("../data/liver_status_new.csv", index=False)